In [280]:
import pandas as pd 
import numpy as np
from collections import Counter
from collections import OrderedDict

import warnings
warnings.filterwarnings('ignore')

In [298]:
df_train = pd.read_csv('./coursera_sessions_train.txt', sep=';', header=-1, names=['look', 'buy'])
df_test = pd.read_csv('./coursera_sessions_test.txt', sep=';', header=-1, names=['look', 'buy'])

In [299]:
# learn freq dictionary by train sample 

counter1 = Counter()
counter2 = Counter()

for sess in df_train.itertuples():
    
    look = sess[1]
    buy = sess[2]
    
    if isinstance(look, str):
        counter1.update(look.split(','))
    if isinstance(buy, str):
        counter2.update(buy.split(','))
        
look_freq = OrderedDict(sorted(counter1.items(), key=lambda x:x[1], reverse=True))
buy_freq = OrderedDict(sorted(counter2.items(), key=lambda x:x[1], reverse=True))        


In [300]:
# convert str to list 

def to_list(sess):
    return sess.split(',')

train = df_train[~pd.isnull(df_train.buy)]
test = df_test[~pd.isnull(df_test.buy)]

train['look'] = train['look'].apply(to_list)
train['buy'] = train['buy'].apply(to_list)
test['look'] = test['look'].apply(to_list)
test['buy'] = test['buy'].apply(to_list)


In [307]:
# session = test.iloc[0, 0]
# freq_dict = look_freq

# items = []
# for item in session:
#     if item not in items:
#         items.append(item)

# # indexes for sorting 

# idx = list(range(len(items)))[::-1]

# # freqs from dictionary 

# freqs = []
# for item in session:
#     if item in freq_dict.keys():
#         freqs.append(freq_dict[item])
#     else:
#         freqs.append(0)

# to_sort = list(zip(items, idx, freqs))
# to_sort
# # rec = sorted(to_sort, key = lambda x: (x[2], x[1]), reverse=True)
# # list(map(lambda x: x[0], rec)), look_freq['66'], look_freq


[('63', 6, 6),
 ('68', 5, 2),
 ('69', 4, 0),
 ('70', 3, 0),
 ('66', 2, 2),
 ('61', 1, 2),
 ('59', 0, 1)]

In [308]:
def add_rec(session, freq_dict):
       
    # items list to set     
    
    items = []
    for item in session:
        if item not in items:
            items.append(item)
    
    # indexes for sorting 
    
    idx = list(range(len(items)))[::-1]
    
    # freqs from dictionary 
    
    freqs = []
    for item in session:
        if item in freq_dict.keys():
            freqs.append(freq_dict[item])
        else:
            freqs.append(0)

    to_sort = list(zip(items, idx, freqs))
    rec = sorted(to_sort, key = lambda x: (x[2], x[1]), reverse=True)

    return list(map(lambda x: x[0], rec))

def precision_k(rec, buy, k):
    
    y = 0
    for i_rec in rec[:k]:
        if i_rec in buy:
            y += 1
    return y / k

def recall_k(rec, buy, k):
    y = 0
    for i_rec in rec[:k]:
        if i_rec in buy:
            y += 1
    return y / len(buy)

def write_ans(ls, name):
    with open(name, 'w') as fl:
        fl.write(' '.join(map(str, np.round(ls, 2))))
          
def compute_metrics(sample, dict_rec, name):

    sample['rec'] = sample['look'].apply(lambda x: add_rec(x, dict_rec))
    
    recall1 = sample.apply(lambda x: recall_k(x.rec, x.buy, 1), axis=1)
    precision1 = sample.apply(lambda x: precision_k(x.rec, x.buy, 1), axis=1)
    recall5 = sample.apply(lambda x: recall_k(x.rec, x.buy, 5), axis=1)
    precision5 = sample.apply(lambda x: precision_k(x.rec, x.buy, 5), axis=1)

    metrics = [ 
        recall1.sum() / train.shape[0], 
        precision1.sum() / train.shape[0], 
        recall5.sum() / train.shape[0], 
        precision5.sum() / train.shape[0], 
    ]

    write_ans(metrics, name)
    

In [309]:
compute_metrics(train, look_freq, 'ans1.txt')
compute_metrics(test, look_freq, 'ans2.txt')
# compute_metrics(train, buy_rec, 'ans3.txt')
# compute_metrics(test, buy_rec, 'ans4.txt')